In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import cvxpy as cp

from manipulation.manipulation import Manipulation
import pickle, os

In [2]:
#load train/test data
prob = Manipulation() #use default config, pass different config file oth.
config_fn = './manipulation/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/manipulation/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
train_data = pickle.load(train_file)
train_file.close()

test_file = open(dataset_fn+'/test.p','rb')
test_data = pickle.load(test_file)
p_test, as_test, fs_test, y_test, c_test, times_test = test_data
test_file.close()

n_test = len(c_test)

# test MLOPT solver

In [3]:
from solvers.mlopt import MLOPT

system = 'manipulation'
prob_features = ['w', 'mu']
mlopt_obj = MLOPT(system, prob, prob_features)

n_features = 13
mlopt_obj.construct_strategies(n_features, train_data)
print(mlopt_obj.n_strategies)

9


In [4]:
mlopt_obj.setup_network()

fn_saved = 'models/mlopt_manipulation_20200709_1735.pt'
mlopt_obj.load_network(fn_saved)

mlopt_obj.model

FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=13, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=9, bias=True)
  )
)

In [6]:
mlopt_obj.training_params['TRAINING_ITERATIONS'] = 1000
mlopt_obj.train(verbose=False)
print(mlopt_obj.model_fn)

Saved model at /home/acauligi/cs_234/project/mlopt-micp/mlopt_manipulation_20200709_1735.pt
Done training
/home/acauligi/cs_234/project/mlopt-micp/mlopt_manipulation_20200709_1735.pt


In [10]:
n_succ = 0
count = 0

costs_mlopt = []
total_time_mlopt = []
num_solves_mlopt = []

cost_ratios_mlopt = []
costs_ip = []
total_time_ip = []
num_solves_ip = []

for ii in range(n_test):
    if ii % 1000 == 0:
        print('{} / {}'.format(ii,n_test))
    prob_params = {}
    for k in p_test.keys():
        prob_params[k] = p_test[k][ii]

    try:
        prob_success, cost, total_time, n_evals = mlopt_obj.forward(prob_params)

        count += 1
        if prob_success:
            n_succ += 1
            costs_mlopt += [cost]
            total_time_mlopt += [total_time]
            num_solves_mlopt += [n_evals]

            true_cost = c_test[ii]
            costs_ip += [true_cost]
            total_time_ip += [times_test[ii]]

            cost_ratios_naive_mlopt += [cost / true_cost]
    except:
        print('failed')
        continue

costs_mlopt = np.array(costs_mlopt)
cost_ratios_mlopt = np.array(cost_ratios_mlopt)
total_time_mlopt = np.array(total_time_mlopt)
num_solves_mlopt = np.array(num_solves_mlopt, dtype=int)

costs_ip = np.array(costs_ip)
total_time_ip = np.array(total_time_ip)
num_solves_ip = np.array(num_solves_ip, dtype=int)

mlopt_percentage = 100 * float(n_succ) / float(count)
mlopt_percentage

0 / 500
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
failed
faile

ZeroDivisionError: float division by zero

# test Regression solver

In [7]:
from solvers.regression import Regression

system = 'cartpole'
prob_features = ['w', 'mu']
reg_obj = Regression(system, prob, prob_features)

n_features = 13
reg_obj.construct_strategies(n_features, train_data)

In [8]:
reg_obj.setup_network()

fn_saved = 'models/regression_cartpole_20200709_1751.pt'
reg_obj.load_network(fn_saved)

reg_obj.model

FFNet(
  (activation): ReLU()
  (layers): ModuleList(
    (0): Linear(in_features=13, out_features=32, bias=True)
    (1): Linear(in_features=32, out_features=32, bias=True)
    (2): Linear(in_features=32, out_features=32, bias=True)
    (3): Linear(in_features=32, out_features=30, bias=True)
  )
)

In [9]:
reg_obj.training_params['TRAINING_ITERATIONS'] = 1000
reg_obj.train(verbose=False)
print(reg_obj.model_fn)

Saved model at /home/acauligi/cs_234/project/mlopt-micp/regression_cartpole_20200709_1751.pt
Done training
/home/acauligi/cs_234/project/mlopt-micp/regression_cartpole_20200709_1751.pt


In [ ]:
n_succ = 0
count = 0

costs_reg = []
total_time_reg = []
num_solves_reg = []

cost_ratios_reg = []

for ii in range(n_test):
    prob_params = {}
    p_test = test_data[0]
    for k in p_test.keys():
        prob_params[k] = p_test[k][ii]

    try:
        prob_success, cost, total_time = reg_obj.forward(prob_params)

        count += 1
        if prob_success:
            n_succ += 1
            costs_reg += [cost]
            total_time_reg += [total_time]
            num_solves_reg += [1]

            true_cost = c_test[ii]
            cost_ratios_reg += [cost / true_cost]
    except:
        continue

costs_reg = np.array(costs_reg)
cost_ratios_reg = np.array(cost_ratios_reg)
total_time_reg = np.array(total_time_reg)
num_solves_reg = np.array(num_solves_reg, dtype=int)

reg_percentage = 100 * float(n_succ) / float(count)
reg_percentage